In [1]:
from pathlib import Path
import copy

import torch
import pyscf
from pyscf import dft
import numpy as np

from cadft.utils import NAO
from cadft.utils import FCNet
from cadft import CC_DFT_DATA, Mol

/home/dhem/anaconda3/envs/pyscf/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecular = copy.deepcopy(Mol["Methane"])
molecular[0][1] += 0

dft2cc = CC_DFT_DATA(
    molecular,
    name="test",
    basis="cc-pvdz",
    if_basis_str=True,
)

mf = pyscf.scf.RHF(dft2cc.mol)
mf.kernel()
mycc = pyscf.cc.CCSD(mf)
mycc.kernel()

dm1_cc = mycc.make_rdm1(ao_repr=True)
e_cc = mycc.e_tot

mdft = pyscf.scf.RKS(dft2cc.mol)
mdft.xc = "b3lyp"
mdft.kernel()
dm1_dft = mdft.make_rdm1(ao_repr=True)

In [4]:
from cadft.utils import (
    add_args,
    gen_keys_l,
    gen_model_dict,
    load_model,
)

key_l = []
model_dict = {}

ATOM_LIST = [
    "H",
    "C",
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

keys_l = gen_keys_l(ATOM_LIST)
model_dict = gen_model_dict(keys_l, 800, device)
load_model(model_dict, keys_l, "2024-05-18-19-35-45", 800, device)

Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-H-D-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-H-D-2-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-H-O-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-H-O-2-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-C-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/H-C-2-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-H-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-H-2-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-C-D-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-C-D-2-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-C-O-1-0.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-18-19-35-45-800/C-

In [5]:
def gen_dm1(dft2cc, molecular, dm1, model_dict, device):
    dm1_cc = np.zeros((dft2cc.mol.nao, dft2cc.mol.nao))
    for i in range(dft2cc.mol.natm):
        for j in range(dft2cc.mol.natm):
            if molecular[i][0] != molecular[j][0]:
                key = f"{molecular[i][0]}-{molecular[j][0]}"
            else:
                if i == j:
                    key = f"{molecular[i][0]}-{molecular[j][0]}-D"
                else:
                    key = f"{molecular[i][0]}-{molecular[j][0]}-O"
            input_mat = (
                dm1[dft2cc.atom_info["slice"][i], dft2cc.atom_info["slice"][j]]
            ).flatten()
            input_mat = (
                torch.as_tensor(input_mat.copy())
                .to(torch.float64)
                .contiguous()
                .to(device=device)
                .requires_grad_(True)
            )
            output_mat = model_dict[key + "1"](input_mat)
            dm1_cc[dft2cc.atom_info["slice"][i], dft2cc.atom_info["slice"][j]] = (
                output_mat.detach()
                .cpu()
                .numpy()
                .reshape(
                    NAO[dft2cc.atom_info["atom"][i]], NAO[dft2cc.atom_info["atom"][j]]
                )
            )

    return dm1_cc


def gen_f_mat(dft2cc, dm1_cc, model_dict, device):
    f_mat = np.zeros((dft2cc.mol.nao, dft2cc.mol.nao))
    ene_xc = 0
    for i in range(dft2cc.mol.natm):
        for j in range(dft2cc.mol.natm):
            if molecular[i][0] != molecular[j][0]:
                key = f"{molecular[i][0]}-{molecular[j][0]}"
            else:
                if i == j:
                    key = f"{molecular[i][0]}-{molecular[j][0]}-D"
                else:
                    key = f"{molecular[i][0]}-{molecular[j][0]}-O"
            input_mat = (
                dm1_cc[dft2cc.atom_info["slice"][i], dft2cc.atom_info["slice"][j]]
            ).flatten()
            input_mat = (
                torch.as_tensor(input_mat.copy())
                .to(torch.float64)
                .contiguous()
                .to(device=device)
                .requires_grad_(True)
            )
            e_xc = model_dict[key + "2"](input_mat)
            grad_dms = torch.autograd.grad(e_xc.sum(), input_mat)
            # print(e_xc, grad_dms)
            f_mat[dft2cc.atom_info["slice"][i], dft2cc.atom_info["slice"][j]] = (
                grad_dms[0]
                .detach()
                .cpu()
                .numpy()
                .reshape(
                    NAO[dft2cc.atom_info["atom"][i]], NAO[dft2cc.atom_info["atom"][j]]
                )
            )
            ene_xc += e_xc.sum().detach().cpu().numpy()

    return f_mat, ene_xc


dm1_predict = gen_dm1(dft2cc, molecular, dm1_dft, model_dict, device) + dm1_dft
print(np.mean(np.abs(dm1_predict - dm1_cc)))
f_mat, ene_xc = gen_f_mat(dft2cc, dm1_predict, model_dict, device)
h1e = dft2cc.mol.intor("int1e_nuc") + dft2cc.mol.intor("int1e_kin")
eri = dft2cc.mol.intor("int2e")

coords = mdft.grids.coords
weights = mdft.grids.weights
ao_value = dft.numint.eval_ao(dft2cc.mol, coords, deriv=1)

rho = dft.numint.eval_rho(dft2cc.mol, ao_value, dm1_predict, xctype="GGA")
exc_cc_grids = dft.libxc.eval_xc("b3lyp", rho)[0]
ek_mat_cc = np.einsum("pqrs,pr,qs->qs", eri, dm1_predict, dm1_predict)
exc_cc = (
    np.einsum("i,i,i->", exc_cc_grids, rho[0], weights)
    - np.sum(ek_mat_cc) * 0.05
)
e_dft = (
    exc_cc
    + np.einsum("pqrs,pq,rs", eri, dm1_predict, dm1_predict) / 2
    + np.sum(h1e * dm1_predict)
    + dft2cc.mol.energy_nuc()
)

print(ene_xc + 1000 * (e_dft - e_cc))

0.00015059191529139204
-4.777271707317153


In [6]:
print(np.mean(np.abs(dm1_dft - dm1_cc)))

0.004585176802917454


In [10]:
import scipy.linalg as LA

mat_s = dft2cc.mol.intor("int1e_ovlp")
mat_hs = LA.fractional_matrix_power(mat_s, -0.5).real
dm1_scf = dm1_cc.copy()

for _ in range(100):
    fock_mat = mdft.get_fock(dm=dm1_scf) 
    f_mat, ene_xc = gen_f_mat(dft2cc, dm1_scf, model_dict, device)
    fock_mat += f_mat / 1000
    fock_a = mat_hs @ (fock_mat) @ mat_hs
    _, mo = np.linalg.eigh(fock_a)
    mo = mat_hs @ mo
    dm1_old = dm1_scf.copy()
    dm1_scf = 2 * mo[:, : dft2cc.mol.nelec[0]] @ mo[:, : dft2cc.mol.nelec[0]].T
    dm1_scf = 0.05 * dm1_scf + 0.95 * dm1_old
    print(np.mean(np.abs(dm1_old - dm1_scf)), np.mean(np.abs(dm1_cc - dm1_scf)))

0.04825836052809293 0.04825836052809293
0.022813700696431745 0.06314939629534469
0.020108800827317846 0.07693757104526357
0.024623462075746394 0.09557993842897801
0.01734661763608495 0.10013260408196116
0.01596283440582332 0.10471708490110879
0.012828751897542364 0.1068726207577057
0.007474435866596316 0.10486034780036141
0.005382641106905173 0.10083817524067393
0.005050982771069823 0.09739246164695468
0.0049696978732088485 0.09389912814117336
0.00471512862413235 0.09056757810410111
0.004640869964608124 0.0881284076362736
0.004553399510003932 0.08593947806318591
0.01263301665018613 0.08760779611324633
0.01694971696935698 0.08982407667453093
0.03441933548995261 0.10107256485690708
0.027301028145595194 0.1169782516608158
0.021503933365215402 0.12577580622221027
0.01816989202795376 0.12631347401671802
0.01566559639070701 0.12564722463014688
0.018309718764310333 0.1320130836082348
0.026107646179887218 0.14155417712439616
0.006685236183838528 0.13600483154044685
0.0061115251199312095 0.1307

In [19]:
print(np.einsum("pqrs,pq,rs", eri, dm1_predict, dm1_predict))
dm1_predict_flatten = dm1_predict.flatten()
eri_flatten = eri.reshape(
    dft2cc.mol.nao * dft2cc.mol.nao, dft2cc.mol.nao * dft2cc.mol.nao
)
print(dm1_predict_flatten.shape, eri.shape)
dm1_predict_flatten @ eri_flatten @ dm1_predict_flatten

64.97823500818082
(1156,) (34, 34, 34, 34)


64.97823500818089

In [8]:
# print(np.einsum("pqrs,pr->qs", eri, dm1_cc))
# print(f_mat)

In [3]:
import scipy.linalg as LA
import opt_einsum as oe

mat_s = dft2cc.mol.intor("int1e_ovlp")
mat_hs = LA.fractional_matrix_power(mat_s, -0.5).real
nocc = dft2cc.mol.nelec[0]

mdft = pyscf.scf.RKS(dft2cc.mol)
mdft.grids.build()
ao = pyscf.dft.numint.eval_ao(dft2cc.mol, mdft.grids.coords)

dm1 = dm1_cc.copy()
for i in range(100):
    f_mat, ene_xc = gen_f_mat(dft2cc, dm1, model_dict, device)
    vj = np.einsum("pqrs,pq->rs", eri, dm1)
    fock_a = mat_hs @ (h1e + vj + f_mat) @ mat_hs
    _, mo = np.linalg.eigh(fock_a)
    mo = mat_hs @ mo
    dm1_old = dm1.copy()
    dm1 = 2 * mo[:, :nocc] @ mo[:, :nocc].T
    dm1 = 0.01 * dm1 + 0.99 * dm1_old
    
    dm_cc_r = oe.contract(
        "" "uv,gu,gv,g->g",
        dm1_cc,
        ao,
        ao,
        mdft.grids.weights,
        optimize="auto",
    )

    dm_cc_real_r = oe.contract(
        "uv,gu,gv,g->g",
        dm1,
        ao,
        ao,
        mdft.grids.weights,
        optimize="auto",
    )

    nn_ene = (
        ene_xc
        + np.einsum("pqrs,pq,rs", eri, dm1, dm1) / 2
        + np.sum(h1e * dm1)
        + dft2cc.mol.energy_nuc()
    )
    print(
        f"CCSD energy: {e_cc}",
        f"NN energy: {nn_ene}",
    )
    print(np.sum(np.abs(dm_cc_r - dm_cc_real_r)))

    print(np.sum(dm_cc_r * mdft.grids.coords[:, 0]))
    print(np.sum(dm_cc_real_r * mdft.grids.coords[:, 0]))
    print(np.sum(dm_cc_r * mdft.grids.coords[:, 1]))
    print(np.sum(dm_cc_real_r * mdft.grids.coords[:, 1]))
    print(np.sum(dm_cc_r * mdft.grids.coords[:, 2]))
    print(np.sum(dm_cc_real_r * mdft.grids.coords[:, 2]))

CCSD energy: -40.14735754380377 NN energy: -39.99770520302959
0.06648199446414402
-1.49741330446318e-14
0.006504791491696665
6.748941683287768e-15
0.009288352065045372
-1.0473392986209973e-16
-0.008936148740616259
CCSD energy: -40.14735754380377 NN energy: -40.62979335343853
0.13036053589518118
-1.49741330446318e-14
0.00839967706960866
6.748941683287768e-15
0.018042449114605584
-1.0473392986209973e-16
-0.011948329206819255
CCSD energy: -40.14735754380377 NN energy: -41.24523971318373
0.19210619681395252
-1.49741330446318e-14
0.007097667342143496
6.748941683287768e-15
0.01808562709090069
-1.0473392986209973e-16
-0.018414730613095863
CCSD energy: -40.14735754380377 NN energy: -41.73283541981494
0.2437905905532322
-1.49741330446318e-14
0.015351934646489207
6.748941683287768e-15
0.008291055707002478
-1.0473392986209973e-16
-0.022802561683449785
CCSD energy: -40.14735754380377 NN energy: -42.49196705035505
0.2889631637400022
-1.49741330446318e-14
0.03048984783992817
6.748941683287768e-15
0.